# Assignment 1
Halka Hanna

Selecting 3 time series datasets

In [ ]:
import pandas as pd
import os
import warnings

warnings.filterwarnings("ignore")

## Common Helper Functions

Check on null values in columns.

In [ ]:
def is_non_null(df):
    cols_with_missing = [col for col in df.columns if df[col].isnull().any()]
    print('number of missing columns: ', len(cols_with_missing))

Check on data for time series.

In [ ]:
def is_date_valid(df_col, step):
    expected = pd.date_range(df_col.min(), df_col.max(), freq=step)
    missing = expected.difference(df_col)
    print(missing)

## Minimum Temperatures

Daily Minimum Temperatures in Melbourne. The dataset collect data from `1981-01-01` to `1990-12-31`.

| Date       | mean-tempt |
|-------------|------------|
| 1981-01-01  | 20.7       |
| 1981-01-02  | 17.9       |
| 1981-01-03  | 18.8       |
| ...         | ...        |
| 1990-12-31  | 13.0       |

3650 rows × 2 columns

Data from Kaggle: https://www.kaggle.com/datasets/paulbrabban/daily-minimum-temperatures-in-melbourne

In [ ]:
df_tempt = pd.read_csv('data/daily-minimum-temperatures-in-me.csv')
df_tempt = df_tempt.rename(columns={'Daily minimum temperatures in Melbourne, Australia, 1981-1990': 'mean-tempt'})
df_tempt['Date'] = pd.to_datetime(df_tempt['Date'])
df_tempt = df_tempt.sort_values(['Date'])
is_non_null(df_tempt)
is_date_valid(df_tempt['Date'], 'D')

Since we don't have `1984-12-31`, `1988-12-31` we will add it by hands, mean temperature will be the average of nearest neighbor.

In [ ]:
dates = ['1984-12-30', '1985-01-01', '1988-12-30', '1989-01-01']
helper_dates = df_tempt[df_tempt['Date'].isin(dates)]

first_average = (16.4 + 13.3) / 2
second_average = (14.1 + 14.3) / 2

helper_df = pd.DataFrame({
    'Date': ['1984-12-31', '1988-12-31'],
    'mean-tempt': [first_average, second_average]
})

df_tempt = pd.concat([df_tempt, helper_df], ignore_index=True)
df_tempt['Date'] = pd.to_datetime(df_tempt['Date'])
df_tempt = df_tempt.sort_values(['Date'])
is_date_valid(df_tempt['Date'], 'D')

And last but not least duplicates.

In [ ]:
duplicates = df_tempt[df_tempt.duplicated(subset=['Date'])]
duplicates


## Trends in CO2

Weekly $CO_2$ average. The dataset collect data from `1974-05-19` to `2025-10-12`.

| year | month | day | decimal   | average | ndays | 1 year ago | 10 years ago | increase since 1800 |
|------|--------|-----|-----------|----------|--------|-------------|---------------|----------------------|
| 1974 | 5      | 19  | 1974.3795 | 333.37   | 5      | -999.99     | -999.99       | 50.40                |
| 1974 | 5      | 26  | 1974.3986 | 332.95   | 6      | -999.99     | -999.99       | 50.06                |
| 1974 | 6      | 2   | 1974.4178 | 332.35   | 5      | -999.99     | -999.99       | 49.60                |
| ...  | ...    | ... | ...       | ...      | ...    | ...         | ...           | ...                  |
| 2025 | 10     | 12  | 2025.7795 | 424.81   | 6      | 422.54      | 398.46        | 148.18               |

---

Our date now has format:

| year | month | day |
|------|--------|-----|
| 1974 | 5      | 19  |
| 1974 | 5      | 26  |
| 1974 | 6      | 2   |
| ...  | ...    | ... |
| 2025 | 10     | 12  |

But we need one column with date time format, so we combine them into one column `datetime`.

| datetime   |
|------------|
| 1974-05-19 |
| 1974-05-26 |
| 1974-06-02 |
| ...        |
| 2025-10-12 |

---

So our table look like:

| datetime   | decimal   | average | ndays | 1 year ago | 10 years ago | increase since 1800 |
|------------|-----------|----------|--------|-------------|---------------|----------------------|
| 1974-05-19 | 1974.3795 | 333.37   | 5      | -999.99     | -999.99       | 50.40                |
| 1974-05-26 | 1974.3986 | 332.95   | 6      | -999.99     | -999.99       | 50.06                |
| 1974-06-02 | 1974.4178 | 332.35   | 5      | -999.99     | -999.99       | 49.60                |
| ...        | ...       | ...      | ...    | ...         | ...           | ...                  |
| 2025-10-12 | 2025.7795 | 424.81   | 6      | 422.54      | 398.46        | 148.18               |

2683 rows × 7 columns

Data from Global Monitoring Laboratory: https://gml.noaa.gov/ccgg/trends/data.html?utm_source=chatgpt.com

In [ ]:
df_co2 = pd.read_csv('data/co2_weekly_mlo.csv')
df_co2['datetime'] = df_co2['year'].astype(str) + "-" + df_co2['month'].astype(str) + "-" + df_co2['day'].astype(str)
df_co2 = df_co2[['datetime', 'decimal',	'average', 'ndays', '1 year ago', '10 years ago', 'increase since 1800']]
df_co2['datetime'] = pd.to_datetime(df_co2['datetime'])
df_co2 = df_co2.sort_values(['datetime'])
is_non_null(df_co2)
is_date_valid(df_co2['datetime'], 'W')

print('\n Duplicates')
duplicates = df_co2[df_co2.duplicated(subset=['datetime'])]
duplicates

In [ ]:
df_co2[df_co2['average'] == -999.99]

In [ ]:
import pandas as pd

df_co2['average'] = df_co2['average'].replace(-999.99, pd.NA)
mask = df_co2['average'] == 'null'

indices = df_co2[mask].index

for i in indices:
    if i > 0 and i < len(df_co2) - 1:
        prev_val = df_co2.loc[i - 1, 'average']
        next_val = df_co2.loc[i + 1, 'average']
        df_co2.loc[i, 'average'] = (prev_val + next_val) / 2


In [ ]:
df_co2[df_co2['average'] == -999.99]

In [ ]:
df_co2[df_co2['average'] == pd.NA]

## Energy Consumption

Hourly Energy Consumption. The dataset collect data from `1981-01-01 01:00:00` to `1990-12-31 00:00:00`.

| Datetime            | AEP_MW  |
|---------------------|---------|
| 2004-10-31 01:00:00 | 12379.0 |
| 2004-10-31 02:00:00 | 11935.0 |
| 2004-10-31 03:00:00 | 11692.0 |
| ...                 | ...     |
| 2018-08-03 00:00:00 | 14809.0 |

121273 rows × 2 columns

Data from Kaggle: https://www.kaggle.com/datasets/robikscube/hourly-energy-consumption?utm_source=chatgpt.com

In [ ]:
df_aep = pd.read_csv('data/AEP_hourly.csv')
df_aep['Datetime'] = pd.to_datetime(df_aep['Datetime'])
df_aep = df_aep.sort_values(['Datetime'])
is_non_null(df_aep)
is_date_valid(df_aep['Datetime'], 'H')

Hear we have a lot of missing data. Since we don't have much time we will use simple mean but we will fixe it in future.

In [ ]:
helper_df = pd.DataFrame({
    'Datetime': ['2004-10-31 02:00:00', '2005-04-03 03:00:00',
             '2005-10-30 02:00:00', '2006-04-02 03:00:00',
             '2006-10-29 02:00:00', '2007-03-11 03:00:00',
             '2007-11-04 02:00:00', '2008-03-09 03:00:00',
             '2008-11-02 02:00:00', '2009-03-08 03:00:00',
             '2009-11-01 02:00:00', '2010-03-14 03:00:00',
             '2010-11-07 02:00:00', '2010-12-10 00:00:00',
             '2011-03-13 03:00:00', '2011-11-06 02:00:00',
             '2012-03-11 03:00:00', '2012-11-04 02:00:00',
             '2012-12-06 04:00:00', '2013-03-10 03:00:00',
             '2013-11-03 02:00:00', '2014-03-09 03:00:00',
             '2014-03-11 14:00:00', '2015-03-08 03:00:00',
             '2016-03-13 03:00:00', '2017-03-12 03:00:00',
             '2018-03-11 03:00:00'],
    'AEP_MW': 0
})

df_aep = pd.concat([df_aep, helper_df], ignore_index=True)
df_aep['Datetime'] = pd.to_datetime(df_aep['Datetime'])
df_aep = df_aep.sort_values('Datetime')

missing = pd.to_datetime([
    '2004-10-31 02:00:00','2005-04-03 03:00:00','2005-10-30 02:00:00',
    '2006-04-02 03:00:00','2006-10-29 02:00:00','2007-03-11 03:00:00',
    '2007-11-04 02:00:00','2008-03-09 03:00:00','2008-11-02 02:00:00',
    '2009-03-08 03:00:00','2009-11-01 02:00:00','2010-03-14 03:00:00',
    '2010-11-07 02:00:00','2010-12-10 00:00:00','2011-03-13 03:00:00',
    '2011-11-06 02:00:00','2012-03-11 03:00:00','2012-11-04 02:00:00',
    '2012-12-06 04:00:00','2013-03-10 03:00:00','2013-11-03 02:00:00',
    '2014-03-09 03:00:00','2014-03-11 14:00:00','2015-03-08 03:00:00',
    '2016-03-13 03:00:00','2017-03-12 03:00:00','2018-03-11 03:00:00'
])

df = df_aep.sort_values('Datetime').set_index('Datetime')
df2 = df.reindex(df.index.union(missing)).sort_index()

prev = df2['AEP_MW'].ffill()
next = df2['AEP_MW'].bfill()

avg_at_missing = ((prev + next) / 2).loc[missing]

df2.loc[missing, 'AEP_MW'] = avg_at_missing
df_filled = df2.reset_index()

df_aep = df_filled.copy()
df_aep = df_aep.rename(columns={'index': 'Datetime'})
is_non_null(df_aep)
is_date_valid(df_aep['Datetime'], 'H')

And duplicates, hear we have four duplicates. For now we will just delete them.

In [ ]:
print('\n Duplicates')
duplicates = df_aep[df_aep.duplicated(subset=['Datetime'])]
duplicates

In [ ]:
df_aep = df_aep.drop_duplicates(subset=['Datetime'])
print('\n Duplicates')
duplicates = df_aep[df_aep.duplicated(subset=['Datetime'])]
duplicates

In [ ]:
df_aep[df_aep['AEP_MW'] == 0]

In [ ]:
import pandas as pd

df_aep['AEP_MW'] = df_aep['AEP_MW'].replace(0, pd.NA)
mask = df_aep['AEP_MW'] == 'null'

indices = df_aep[mask].index

for i in indices:
    if i > 0 and i < len(df_aep) - 1:
        prev_val = df_co2.loc[i - 1, 'AEP_MW']
        next_val = df_co2.loc[i + 1, 'AEP_MW']
        df_co2.loc[i, 'AEP_MW'] = (prev_val + next_val) / 2

In [ ]:
df_aep[df_aep['AEP_MW'] == 0]

## Save to files

In [ ]:
os.makedirs('../data', exist_ok=True)

df_tempt.to_csv('../data/daily_tempt.csv', index=False)
df_co2.to_csv('../data/weekly_co2.csv', index=False)
df_aep.to_csv('../data/hourly_aep.csv', index=False)